In [1]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
    tree = Et.parse(filexml)
    root = tree.getroot()
    samples = []
    for i in range(0, len(root)):
        sample = {'result': []}
        for j, e in enumerate(root[i]):
            if e.tag == "t1":
                sample['result'] = format_first_line(e.text.strip())
            elif e.tag == "t2":
                question = e.text.strip()
                sample['content'] = question if len(question) > 0 else None
        sample.update(
            {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
        # filter the noise samples
        if sample['content'] is not None:
            samples.append(sample)
        else:
            print("[Important warning] samples {} is ignored".format(sample))
    return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

In [2]:
zeroShot = "According to the given legal reasoning approach.\nDocument: {{premise}}\nQuestion: {{hypothesis}}? True or False"
import jsonlines

train_path = get_all_files_from_path("../data/COLIEE2024statute_data-English/train")
out_trainpath = "../data/COLIEE2024statute_data-English/train.jsonl"

test_path = get_all_files_from_path("../data/COLIEE2024statute_data-English/test")
out_testpath = "../data/COLIEE2024statute_data-English/"
def xml2json(files, out_path):
    result = []
    for file in files:
        data = load_samples(file)
        for k in data:
            item = {}
            if 'index' not in k:
                print(k)
            item.update({"id": k['index']})
            item.update({"content": zeroShot.replace("{{premise}}", k["result"]).replace("{{hypothesis}}", k["content"])})
            if k["label"] == "Y": item.update({"label": "true"})
            else: item.update({"label": "false"})
            result.append(item)

    if "R0" in files[0]:
        out_path = out_path+file.split("/")[-1].replace(".xml", "")+".jsonl"
    with jsonlines.open(out_path, 'w') as writer:
        writer.write_all(result)

xml2json(train_path, out_trainpath)
xml2json([test_path[0]], out_testpath)
xml2json([test_path[1]], out_testpath)
xml2json([test_path[2]], out_testpath)
xml2json([test_path[3]], out_testpath)


In [3]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train":"../data/COLIEE2024statute_data-English/train.jsonl", 
                                          "test1": "../data/COLIEE2024statute_data-English/riteval_R01_en.jsonl",
                                          "test2": "../data/COLIEE2024statute_data-English/riteval_R02_en.jsonl",
                                          "test3": "../data/COLIEE2024statute_data-English/riteval_R03_en.jsonl",
                                          "test4": "../data/COLIEE2024statute_data-English/riteval_R04_en.jsonl"})

print(dataset)


/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 695 examples [00:00, 47449.97 examples/s]
Generating test1 split: 111 examples [00:00, 36676.20 examples/s]
Generating test2 split: 81 examples [00:00, 25255.62 examples/s]
Generating test3 split: 109 examples [00:00, 33073.80 examples/s]
Generating test4 split: 101 examples [00:00, 34949.65 examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 695
    })
    test1: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 111
    })
    test2: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 81
    })
    test3: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 109
    })
    test4: Dataset({
        features: ['id', 'content', 'label'],
        num_rows: 101
    })
})


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-xxl"

# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/home/congnguyen/drive/.cache")

In [3]:
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["content"], truncation=True), batched=True, remove_columns=["content", "label"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 99))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=["content", "label"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 5))
print(f"Max target length: {max_target_length}")


NameError: name 'dataset' is not defined

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Classification: " + item for item in sample["content"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["content", "label", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")
# save datasets to disk for later easy loading
tokenized_dataset["train"].save_to_disk("../output/finetuned/data/train")
tokenized_dataset["test"].save_to_disk("../output/finetuned/data/eval")


In [7]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id = "philschmid/flan-t5-xxl-sharded-fp16"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto", cache_dir="/home/congnguyen/drive/.cache")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:19<00:00,  1.62s/it]


In [8]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_int8_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/peft/utils/other.py:141: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 18,874,368 || all params: 11,154,206,720 || trainable%: 0.16921300163961817


In [9]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


In [10]:
import os
os.environ["WANDB_PROJECT"] = "finetuned-flan-t5-xxl"  # name your W&B project
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

In [11]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="../output/finetuned/lora-flan-t5-xxl"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	# auto_find_batch_size=True,
    per_device_train_batch_size=1,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="wandb",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [12]:
# train model
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sssnowkool (nhmcong). Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float

Step,Training Loss


KeyboardInterrupt: 

In [3]:
# Save our LoRA model & tokenizer results
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

NameError: name 'trainer' is not defined

In [1]:
# from datasets import load_dataset

# train_dataset = load_dataset("json", data_files="../data/COLIEE2024statute_data-English/data_full.jsonl", split='train[0:900]')
# test_dataset = load_dataset("json", data_files="../data/COLIEE2024statute_data-English/data_full.jsonl", split='train[900:-1]')
# print(test_dataset)
# print(train_dataset)

/home/s2210436/envs/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'content', 'label'],
    num_rows: 196
})
Dataset({
    features: ['id', 'content', 'label'],
    num_rows: 900
})


In [9]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "../output/finetuned/lora-flan-t5-xxl_batch?auto_epochs30/checkpoint-2500"
# peft_model_id = "../output/finetuned/peft_results_batch?auto_epochs30"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path, device_map="auto", 
                                              cache_dir="/home/congnguyen/drive/.cache", load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, cache_dir="/home/congnguyen/drive/.cache")

# Load the Lora model
# model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id, device_map="auto")
model.eval()
print("Peft model loaded")


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:16<00:00,  1.41s/it]


Peft model loaded


In [10]:
from datasets import load_dataset
from random import randrange

result = {}
# Load dataset from the hub and get a sample
# dataset = load_dataset("samsum")
# sample = dataset['test'][randrange(len(dataset["test"]))]
count = 0

# for i in range(len(test_dataset["content"])):
for item in tqdm(dataset["test1"]):
    id = item["id"]
    label = item["label"]
    content = item["content"]
#     # inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
#     # outputs = model.generate(inputs, max_new_tokens=10)
    # =======================
    input_ids = tokenizer(content, return_tensors="pt").input_ids.cuda()
    # outputs = model.generate(input_ids=input_ids, max_new_tokens=10, do_sample=True, top_p=0.0000001)
    outputs = model.generate(input_ids=input_ids, max_new_tokens=10)
    output_text = format_output(tokenizer.decode(outputs[0]).replace(content, "").split("\n")[-1])
    print(output_text)
#     # if tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0] == test_dataset["label"][i] and i > 96:
#     #     count += 1
    answer = "false"
    if  "true" in output_text or "yes" in output_text:
        answer = "true"
    if answer == label:
        count += 1
    if answer == "true": answer = "Y"
    else: answer = "N"
    result.update({id: answer})
        


  1%|█▍                                                                                                                                                         | 1/111 [00:01<02:10,  1.19s/it]

true


  2%|██▊                                                                                                                                                        | 2/111 [00:02<01:59,  1.09s/it]Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


true


  3%|████▏                                                                                                                                                      | 3/111 [00:03<02:25,  1.34s/it]

...

false


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 110/111 [02:02<00:01,  1.10s/it]

false


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [02:03<00:00,  1.11s/it]

false


In [12]:
print(count)
print(count/len(dataset["test1"]))

77
0.6936936936936937


In [18]:
import json

f = open('../data/analysis.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)

check = {}
for k in data['riteval_R04_en.xml']:
    for i in data['riteval_R04_en.xml'][k]:
        check.update({i: 0})
check

{'R04-02-E': 0,
 'R04-04-E': 0,
 'R04-04-O': 0,
 'R04-05-U': 0,
 'R04-09-A': 0,
 'R04-09-I': 0,
 'R04-09-U': 0,
 'R04-10-O': 0,
 'R04-14-A': 0,
 'R04-16-I': 0,
 'R04-16-U': 0,
 'R04-16-O': 0,
 'R04-18-A': 0,
 'R04-18-U': 0,
 'R04-18-O': 0,
 'R04-19-O': 0,
 'R04-20-E': 0,
 'R04-22-I': 0,
 'R04-23-O': 0,
 'R04-25-O': 0,
 'R04-26-I': 0,
 'R04-26-E': 0,
 'R04-27-A': 0,
 'R04-27-I': 0,
 'R04-29-I': 0,
 'R04-29-O': 0,
 'R04-36-U': 0,
 'R04-37-I': 0,
 'total': 0,
 'R04-08-A': 0,
 'R04-08-O': 0,
 'R04-19-E': 0,
 'R04-20-A': 0,
 'R04-20-O': 0,
 'R04-26-U': 0,
 'R04-37-U': 0}

In [22]:
k_result = {}
f = open("../output/fewshot_detail/prompt_4/riteval_R04_en_acc.txt", "r")
label = {}
for line in f:
    line = line.strip()
    line = line.split("\t")
    k_result.update({line[0]: line[1]})
    label.update({line[0]: line[2]})

count = 0
count2 = 0
for k in k_result:
    answer = k_result[k]
    # if k not in check:
    #     # if  result[k] != label[k] and result[k] == k_result[k]:
    #     #     count += 1
    #     # else:
    #     answer = result[k]
    #         # print(k, ": ", result[k], " - " , k_result[k], " - ",label[k]) 
    #         # print("==========================")
    # else:
    #     answer = k_result[k]
    #     # print(k, ": ", result[k], " + ", k_result[k],  " - " ,label[k]) 
    if answer == label[k]:
        count += 1

print(count)
print(count/len(label))

81
0.801980198019802


In [14]:
dataset["test1"][0]

{'id': 'R01-1-A',
 'content': "According to the given legal reasoning approach.\nDocument: Article 5\n(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.\n(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.\n(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. The same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.\nQuestion: A contract of sales concluded by a minor may not be rescinded if it relates to daily life, even in cases the consent of the parental authority is not obtained.? True or 

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

def evaluate_peft_model(sample,max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("../data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

# compute metric
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

# Rogue1: 50.386161%
# rouge2: 24.842412%
# rougeL: 41.370130%
# rougeLsum: 41.394230%
